# Intro

This notebook is to demonstrate how to convert a [libpgm](https://github.com/CyberPoint/libpgm) discrete Bayesian Belief Network (BBN) into a py-bbn BBN one. The JSON data specified [here](https://pythonhosted.org/libpgm/unittestdict.html) is the example taken.

In [1]:
json_data = {
    "V": ["Letter", "Grade", "Intelligence", "SAT", "Difficulty"],
    "E": [["Intelligence", "Grade"],
        ["Difficulty", "Grade"],
        ["Intelligence", "SAT"],
        ["Grade", "Letter"]],
    "Vdata": {
        "Letter": {
            "ord": 4,
            "numoutcomes": 2,
            "vals": ["weak", "strong"],
            "parents": ["Grade"],
            "children": None,
            "cprob": {
                "['A']": [.1, .9],
                "['B']": [.4, .6],
                "['C']": [.99, .01]
            }
        },

        "SAT": {
            "ord": 3,
            "numoutcomes": 2,
            "vals": ["lowscore", "highscore"],
            "parents": ["Intelligence"],
            "children": None,
            "cprob": {
                "['low']": [.95, .05],
                "['high']": [.2, .8]
            }
        },

        "Grade": {
            "ord": 2,
            "numoutcomes": 3,
            "vals": ["A", "B", "C"],
            "parents": ["Difficulty", "Intelligence"],
            "children": ["Letter"],
            "cprob": {
                "['easy', 'low']": [.3, .4, .3],
                "['easy', 'high']": [.9, .08, .02],
                "['hard', 'low']": [.05, .25, .7],
                "['hard', 'high']": [.5, .3, .2]
            }
        },

        "Intelligence": {
            "ord": 1,
            "numoutcomes": 2,
            "vals": ["low", "high"],
            "parents": None,
            "children": ["SAT", "Grade"],
            "cprob": [.7, .3]
        },

        "Difficulty": {
            "ord": 0,
            "numoutcomes": 2,
            "vals": ["easy", "hard"],
            "parents": None,
            "children": ["Grade"],
            "cprob":  [.6, .4]
        }
    }
}

# Conversion

~~Here we kludge how to build a libpgm discrete network. Note that libpgm requires node data and a skeleton. Since I could not find a factory method to read in from a dictionary or JSON string (they only show how to read the JSON data from a file), I took a look at the code and manually constructed a discrete BBN in libpgm. If the libpgm API changes, then this all might break too.~~

Note that we do **NOT** support dependency on `libpgm` anymore since it is not Python 3.x compatible. You will have to get either the JSON string value or the dictionary representation to work with py-bbn. The culprit with libgpm is [this line here](https://github.com/CyberPoint/libpgm/blob/master/libpgm/discretebayesiannetwork.py#L87). After you have the JSON or dictionary specifying a libpgm BBN, you can use the `Factory` to create a py-bbn BBN and perform exact inference as follows.

In [2]:
from pybbn.graph.dag import Bbn
from pybbn.graph.jointree import EvidenceBuilder
from pybbn.pptc.inferencecontroller import InferenceController
from pybbn.graph.factory import Factory

bbn = Factory.from_libpgm_discrete_dictionary(json_data)
join_tree = InferenceController.apply(bbn)

# Inference

These are the marginal probabilities of each node.

In [3]:
# print the marginal probabilities
for node in join_tree.get_bbn_nodes():
    potential = join_tree.get_bbn_potential(node)
    print(node)
    print(potential)
    print('>')

0|Letter|weak,strong
0=weak|0.432044
0=strong|0.567956
>
1|Grade|A,B,C
1=A|0.447
1=B|0.2714
1=C|0.2816
>
2|Intelligence|low,high
2=low|0.7
2=high|0.3
>
3|SAT|lowscore,highscore
3=lowscore|0.725
3=highscore|0.275
>
4|Difficulty|easy,hard
4=easy|0.6
4=hard|0.4
>


These are the marginal probabilities given that we observe `SAT=highscore`.

In [4]:
# insert an observation evidence
ev = EvidenceBuilder() \
    .with_node(join_tree.get_bbn_node_by_name('SAT')) \
    .with_evidence('highscore', 1.0) \
    .build()
join_tree.unobserve_all()
join_tree.set_observation(ev)

# print the marginal probabilities
for node in join_tree.get_bbn_nodes():
    potential = join_tree.get_bbn_potential(node)
    print(node)
    print(potential)
    print('>')

0|Letter|weak,strong
0=weak|0.590735272727
0=strong|0.409264727273
>
1|Grade|A,B,C
1=A|0.269454545455
1=B|0.270254545455
1=C|0.460290909091
>
2|Intelligence|low,high
2=low|0.127272727273
2=high|0.872727272727
>
3|SAT|lowscore,highscore
3=lowscore|0.0
3=highscore|1.0
>
4|Difficulty|easy,hard
4=easy|0.6
4=hard|0.4
>


These are the marginal probabilities given that we observe `SAT=lowscore`.

In [5]:
# insert an observation evidence
ev = EvidenceBuilder() \
    .with_node(join_tree.get_bbn_node_by_name('SAT')) \
    .with_evidence('lowscore', 1.0) \
    .build()
join_tree.unobserve_all()
join_tree.set_observation(ev)

# print the marginal probabilities
for node in join_tree.get_bbn_nodes():
    potential = join_tree.get_bbn_potential(node)
    print(node)
    print(potential)
    print('>')

0|Letter|weak,strong
0=weak|0.371850758621
0=strong|0.628149241379
>
1|Grade|A,B,C
1=A|0.514344827586
1=B|0.271834482759
1=C|0.213820689655
>
2|Intelligence|low,high
2=low|0.91724137931
2=high|0.0827586206897
>
3|SAT|lowscore,highscore
3=lowscore|1.0
3=highscore|0.0
>
4|Difficulty|easy,hard
4=easy|0.6
4=hard|0.4
>
